In [1]:
import torch
from datasets import load_dataset
from torch.utils.data import DataLoader

/mnt/disk1/sumdev/CourseWork/a3_env/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Load training dataset, only the first 50,000 examples
train_dataset = load_dataset("wmt16", "de-en", split="train[:100000]")

# Load validation and test datasets
validation_dataset = load_dataset("wmt16", "de-en", split="validation")
test_dataset = load_dataset("wmt16", "de-en", split="test")
print(len(train_dataset), len(validation_dataset), len(test_dataset))

100000 2169 2999


# Setup-2A

In [3]:
from torch import Tensor
import torch
import torch.nn as nn
from torch.nn import Transformer
import math

DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# helper Module that adds positional encoding to the token embedding to introduce a notion of word order.
class PositionalEncoding(nn.Module):
    def __init__(self,
                 emb_size: int,
                 dropout: float,
                 maxlen: int = 5000):
        super(PositionalEncoding, self).__init__()
        den = torch.exp(- torch.arange(0, emb_size, 2)* math.log(10000) / emb_size)
        pos = torch.arange(0, maxlen).reshape(maxlen, 1)
        pos_embedding = torch.zeros((maxlen, emb_size))
        pos_embedding[:, 0::2] = torch.sin(pos * den)
        pos_embedding[:, 1::2] = torch.cos(pos * den)
        pos_embedding = pos_embedding.unsqueeze(-2)

        self.dropout = nn.Dropout(dropout)
        self.register_buffer('pos_embedding', pos_embedding)

    def forward(self, token_embedding: Tensor):
        return self.dropout(token_embedding + self.pos_embedding[:token_embedding.size(0), :])

# helper Module to convert tensor of input indices into corresponding tensor of token embeddings
class TokenEmbedding(nn.Module):
    def __init__(self, vocab_size: int, emb_size):
        super(TokenEmbedding, self).__init__()
        self.embedding = nn.Embedding(vocab_size, emb_size)
        self.emb_size = emb_size

    def forward(self, tokens: Tensor):
        return self.embedding(tokens.long()) * math.sqrt(self.emb_size)

# Seq2Seq Network
class Seq2SeqTransformer(nn.Module):
    def __init__(self,
                 num_encoder_layers: int,
                 num_decoder_layers: int,
                 emb_size: int,
                 nhead: int,
                 src_vocab_size: int,
                 tgt_vocab_size: int,
                 dim_feedforward: int = 512,
                 dropout: float = 0.1):
        super(Seq2SeqTransformer, self).__init__()
        self.transformer = Transformer(d_model=emb_size,
                                       nhead=nhead,
                                       num_encoder_layers=num_encoder_layers,
                                       num_decoder_layers=num_decoder_layers,
                                       dim_feedforward=dim_feedforward,
                                       dropout=dropout)
        self.generator = nn.Linear(emb_size, tgt_vocab_size)
        self.src_tok_emb = TokenEmbedding(src_vocab_size, emb_size)
        self.tgt_tok_emb = TokenEmbedding(tgt_vocab_size, emb_size)
        self.positional_encoding = PositionalEncoding(
            emb_size, dropout=dropout)

    def forward(self,
                src: Tensor,
                trg: Tensor,
                src_mask: Tensor,
                tgt_mask: Tensor,
                src_padding_mask: Tensor,
                tgt_padding_mask: Tensor,
                memory_key_padding_mask: Tensor):
        src_emb = self.positional_encoding(self.src_tok_emb(src))
        tgt_emb = self.positional_encoding(self.tgt_tok_emb(trg))
        outs = self.transformer(src_emb, tgt_emb, src_mask, tgt_mask, None,
                                src_padding_mask, tgt_padding_mask, memory_key_padding_mask)
        return self.generator(outs)

    def encode(self, src: Tensor, src_mask: Tensor):
        return self.transformer.encoder(self.positional_encoding(
                            self.src_tok_emb(src)), src_mask)

    def decode(self, tgt: Tensor, memory: Tensor, tgt_mask: Tensor):
        return self.transformer.decoder(self.positional_encoding(
                          self.tgt_tok_emb(tgt)), memory,
                          tgt_mask)

In [4]:
from typing import Iterable
import torch
import math
import torch.nn as nn
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator
from torchtext.datasets import Multi30k
from typing import Iterable, List


# Define function to yield tokens from the dataset
def yield_tokens(data_iter: Iterable, language: str):
    for data_sample in data_iter:
        yield token_transform[language](data_sample['translation'][language])

# Set up tokenizers
token_transform = {}
SRC_LANGUAGE = 'de'
TGT_LANGUAGE = 'en'
token_transform[SRC_LANGUAGE] = get_tokenizer('spacy', language='de_core_news_sm')
token_transform[TGT_LANGUAGE] = get_tokenizer('spacy', language='en_core_web_sm')


# Build vocabulary
vocab_transform = {}
UNK_IDX, PAD_IDX, BOS_IDX, EOS_IDX = 0, 1, 2, 3
vocab_transform[SRC_LANGUAGE] = build_vocab_from_iterator(yield_tokens(train_dataset, 'de'), specials=["<unk>", "<pad>", "<bos>", "<eos>"])
vocab_transform[TGT_LANGUAGE] = build_vocab_from_iterator(yield_tokens(train_dataset, 'en'), specials=["<unk>", "<pad>", "<bos>", "<eos>"])
vocab_transform[SRC_LANGUAGE].set_default_index(UNK_IDX)
vocab_transform[TGT_LANGUAGE].set_default_index(UNK_IDX)

In [5]:
def sequential_transforms(*transforms):
    def func(txt_input):
        for transform in transforms:
            txt_input = transform(txt_input)
        return txt_input
    return func

# function to add BOS/EOS and create tensor for input sequence indices
def tensor_transform(token_ids: List[int]):
    return torch.cat((torch.tensor([BOS_IDX]),
                      torch.tensor(token_ids),
                      torch.tensor([EOS_IDX])))
    
text_transform = {}
for ln in [SRC_LANGUAGE, TGT_LANGUAGE]:
    text_transform[ln] = sequential_transforms(token_transform[ln], #Tokenization
                                               vocab_transform[ln], #Numericalization
                                               tensor_transform) # Add BOS/EOS and create tensor

In [6]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

def generate_square_subsequent_mask(sz):
    mask = (torch.triu(torch.ones((sz, sz), device=device)) == 1).transpose(0, 1)
    mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))
    return mask

def create_mask(src, tgt):
    src_seq_len = src.shape[1]
    tgt_seq_len = tgt.shape[1]

    tgt_mask = generate_square_subsequent_mask(tgt_seq_len)
    src_mask = torch.zeros((src_seq_len, src_seq_len),device=device).type(torch.bool)
    src_padding_mask = (src == PAD_IDX)
    tgt_padding_mask = (tgt == PAD_IDX)
    return src_mask, tgt_mask, src_padding_mask, tgt_padding_mask

In [7]:
SRC_LANGUAGE = 'de'
TGT_LANGUAGE = 'en'
vocab_transform = {}
UNK_IDX, PAD_IDX, BOS_IDX, EOS_IDX = 0, 1, 2, 3
vocab_transform[SRC_LANGUAGE] = build_vocab_from_iterator(yield_tokens(train_dataset, 'de'), specials=["<unk>", "<pad>", "<bos>", "<eos>"])
vocab_transform[TGT_LANGUAGE] = build_vocab_from_iterator(yield_tokens(train_dataset, 'en'), specials=["<unk>", "<pad>", "<bos>", "<eos>"])
vocab_transform[SRC_LANGUAGE].set_default_index(UNK_IDX)
vocab_transform[TGT_LANGUAGE].set_default_index(UNK_IDX)

SRC_VOCAB_SIZE = len(vocab_transform[SRC_LANGUAGE])
TGT_VOCAB_SIZE = len(vocab_transform[TGT_LANGUAGE])
EMB_SIZE = 512
NHEAD = 8
FFN_HID_DIM = 512
BATCH_SIZE = 128
NUM_ENCODER_LAYERS = 3
NUM_DECODER_LAYERS = 3

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model_loaded = Seq2SeqTransformer(NUM_ENCODER_LAYERS, NUM_DECODER_LAYERS, EMB_SIZE,
                                 NHEAD, SRC_VOCAB_SIZE, TGT_VOCAB_SIZE, FFN_HID_DIM)
model = model_loaded.to(device)
model_loaded.load_state_dict(torch.load('2A_final_model.pth'))

/mnt/disk1/sumdev/CourseWork/a3_env/lib/python3.12/site-packages/torch/nn/modules/transformer.py:286: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


<All keys matched successfully>

In [8]:
def generate_square_subsequent_mask(sz):
    mask = (torch.triu(torch.ones((sz, sz), device=device)) == 1).transpose(0, 1)
    mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))
    return mask

# function to generate output sequence using greedy algorithm
def greedy_decode(model, src, src_mask, max_len, start_symbol):
    src = src.to(DEVICE)
    src_mask = src_mask.to(DEVICE)

    memory = model.encode(src, src_mask)
    ys = torch.ones(1, 1).fill_(start_symbol).type(torch.long).to(DEVICE)
    for i in range(max_len-1):
        memory = memory.to(DEVICE)
        tgt_mask = (generate_square_subsequent_mask(ys.size(0))
                    .type(torch.bool)).to(DEVICE)
        out = model.decode(ys, memory, tgt_mask)
        out = out.transpose(0, 1)
        prob = model.generator(out[:, -1])
        _, next_word = torch.max(prob, dim=1)
        next_word = next_word.item()

        ys = torch.cat([ys,
                        torch.ones(1, 1).type_as(src.data).fill_(next_word)], dim=0)
        if next_word == EOS_IDX:
            break
    return ys

# actual function to translate input sentence into target language
def translate(model: torch.nn.Module, src_sentence: str):
    model.eval()
    src = text_transform[SRC_LANGUAGE](src_sentence).view(-1, 1)
    num_tokens = src.shape[0]
    src_mask = (torch.zeros(num_tokens, num_tokens)).type(torch.bool)
    tgt_tokens = greedy_decode(
        model,  src, src_mask, max_len=num_tokens + 5, start_symbol=BOS_IDX).flatten()
    return " ".join(vocab_transform[TGT_LANGUAGE].lookup_tokens(list(tgt_tokens.cpu().numpy()))).replace("<bos>", "").replace("<eos>", "")

In [9]:
sentence = 'Obama empfängt Netanyahu'
actual = 'Obama receives Netanyahu'

translate(model_loaded, sentence)

' Is there at present a minimum level of transport'

In [10]:
from sacrebleu.metrics import BLEU
from tqdm import tqdm

def calculate_test_bleu_scores(dataset, model):
    references = []
    hypotheses = []
    bleu = BLEU()
    for example in tqdm(dataset, desc="Translating"):
        src_sentence = example['translation']['de']
        ref_translation = example['translation']['en']

        hyp_translation = translate(model, src_sentence)

        references.append([ref_translation])  # list of possible references is expected by sacrebleu
        hypotheses.append(hyp_translation)

    # Calculate BLEU scores
    bleu_scores = bleu.corpus_score(hypotheses, references)

    # Extract and print BLEU scores for 1-grams to 4-grams
    print(f"BLEU-1: {bleu_scores.precisions[0]}")
    print(f"BLEU-2: {bleu_scores.precisions[1]}")
    print(f"BLEU-3: {bleu_scores.precisions[2]}")
    print(f"BLEU-4: {bleu_scores.precisions[3]}")
    print(f"BLEU score: {bleu_scores.score}")
    return bleu_scores

# Calculating BLEU metrics for the validation corpus
bleu_scores = calculate_test_bleu_scores(validation_dataset, model_loaded)

# Calculating BLEU metrics for the test corpus
bleu_scores = calculate_test_bleu_scores(test_dataset, model_loaded)

Translating:   0%|          | 0/2169 [00:00<?, ?it/s]

Translating: 100%|██████████| 2169/2169 [02:15<00:00, 16.02it/s]


BLEU-1: 100.0
BLEU-2: 71.42857142857143
BLEU-3: 30.76923076923077
BLEU-4: 8.333333333333334
BLEU score: 36.78763249927777


Translating: 100%|██████████| 2999/2999 [03:10<00:00, 15.73it/s]


BLEU-1: 100.0
BLEU-2: 25.0
BLEU-3: 7.142857142857143
BLEU-4: 4.166666666666667
BLEU score: 16.515821590069034


In [12]:
bleu_scores

BLEU = 16.52 100.0/25.0/7.1/4.2 (BP = 1.000 ratio = 1.000 hyp_len = 9 ref_len = 9)

In [13]:
import nltk
nltk.download('wordnet')
nltk.download('omw-1.4')
from nltk.translate.meteor_score import single_meteor_score
from nltk.tokenize import word_tokenize
from tqdm import tqdm
nltk.download('punkt')

def calculate_meteor_score(dataset, model):
    total_score = 0
    num_examples = 0

    for example in tqdm(dataset, desc="Evaluating"):
        src_sentence = example['translation']['de']
        reference = example['translation']['en']
        hypothesis = translate(model, src_sentence)
        reference_tokens = word_tokenize(reference, language='english')
        hypothesis_tokens = word_tokenize(hypothesis, language='english')
        score = single_meteor_score(reference_tokens, hypothesis_tokens)
        total_score += score
        num_examples += 1

    average_score = total_score / num_examples
    return average_score

# Calculate METEOR score for the validation corpus
meteor_score = calculate_meteor_score(validation_dataset, model_loaded)
print(f"Average METEOR Score for validation data: {meteor_score:.4f}")

# Calculate METEOR score for the test corpus
meteor_score = calculate_meteor_score(test_dataset, model_loaded)
print(f"Average METEOR Score for test data: {meteor_score:.4f}")

[nltk_data] Downloading package wordnet to
[nltk_data]     /mnt/disk1/sumdev/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /mnt/disk1/sumdev/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /mnt/disk1/sumdev/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
Evaluating: 100%|██████████| 2169/2169 [02:12<00:00, 16.37it/s]


Average METEOR Score for validation data: 0.3043


Evaluating: 100%|██████████| 2999/2999 [03:07<00:00, 15.97it/s]

Average METEOR Score for test data: 0.3153


In [14]:
from bert_score import score
from tqdm import tqdm
import torch

def calculate_bertscore(dataset, model, device='cuda'):
    # Lists to hold all reference translations and model hypotheses
    references = []
    hypotheses = []
    # Ensure the model is on the correct device
    model.to(device)

    for example in tqdm(dataset, desc="Generating Translations"):
        src_sentence = example['translation']['de']
        reference = example['translation']['en']
        # Generate model's translation
        hypothesis = translate(model, src_sentence)
        references.append(reference)
        hypotheses.append(hypothesis)

    # Compute BERTScore
    P, R, F1 = score(hypotheses, references, lang="en", verbose=True)

    # Calculate average scores
    avg_precision = torch.mean(P).item()
    avg_recall = torch.mean(R).item()
    avg_f1 = torch.mean(F1).item()
    return avg_precision, avg_recall, avg_f1

# Calculate BERTscore on validation corpus
print("BERTscore for validation data: ")
avg_precision, avg_recall, avg_f1 = calculate_bertscore(validation_dataset, model_loaded)
print(f"Average Precision: {avg_precision:.4f}")
print(f"Average Recall: {avg_recall:.4f}")
print(f"Average F1: {avg_f1:.4f}")

# Calculate BERTscore on test corpus
print("BERTscore for test data: ")
avg_precision, avg_recall, avg_f1 = calculate_bertscore(test_dataset, model_loaded)
print(f"Average Precision: {avg_precision:.4f}")
print(f"Average Recall: {avg_recall:.4f}")
print(f"Average F1: {avg_f1:.4f}")

BERTscore for validation data: 


Generating Translations: 100%|██████████| 2169/2169 [02:11<00:00, 16.48it/s]
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


calculating scores...
computing bert embedding.


100%|██████████| 68/68 [00:06<00:00, 10.92it/s]


computing greedy matching.


...

calculating scores...
computing bert embedding.


100%|██████████| 94/94 [00:07<00:00, 12.05it/s]


computing greedy matching.


100%|██████████| 47/47 [00:00<00:00, 161.53it/s]

done in 8.10 seconds, 370.13 sentences/sec
Average Precision: 0.8536
Average Recall: 0.8607
Average F1: 0.8570


# Setup-2B

In [8]:
from transformers import T5Tokenizer, T5ForConditionalGeneration

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
tokenizer = T5Tokenizer.from_pretrained("google-t5/t5-small")
model = T5ForConditionalGeneration.from_pretrained("google-t5/t5-small")
model = model.to(device)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [9]:
import sacrebleu
from nltk.translate import meteor_score
from nltk.tokenize import word_tokenize
from bert_score import score
import nltk
nltk.download('wordnet')
nltk.download('punkt')


def calculate_bleu(predicted_corpus, reference_corpus):
    return sacrebleu.corpus_bleu(predicted_corpus, [[i] for i in reference_corpus])

def calculate_meteor(predicted_corpus, reference_corpus):
    m_score = 0
    for line in zip(predicted_corpus, reference_corpus):
        hypo = word_tokenize(line[0])
        ref = word_tokenize(line[1])
        # m_score += meteor_score.meteor_score([ref], hypo)
        m_score += meteor_score.single_meteor_score(ref, hypo)
    return m_score / len(reference_corpus)

def calculate_bert_score(predicted_corpus, reference_corpus):
    p, r, f1 = score(predicted_corpus, reference_corpus, lang="de")
    return p.mean().item(), r.mean().item(), f1.mean().item()

[nltk_data] Downloading package wordnet to
[nltk_data]     /mnt/disk1/sumdev/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /mnt/disk1/sumdev/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [12]:
## Translate English to German
def translate_engligh_to_german(dataset):
    reference_corpus = []
    predicted_corpus = []
    prefix = "translate English to German: "
    
    for example in dataset:
        input_text = prefix + example["translation"]["en"]
        input_ids = tokenizer(input_text, return_tensors="pt").input_ids
        input_ids = input_ids.to(device)
        outputs = model.generate(input_ids)
        predicted_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

        reference_corpus.append(example["translation"]["de"])
        predicted_corpus.append(predicted_text)
    return reference_corpus, predicted_corpus

In [13]:
reference_corpus_val, predicted_corpus_val = translate_engligh_to_german(validation_dataset)

bleu_score_ = calculate_bleu(predicted_corpus_val, reference_corpus_val)
meteor_score_ = calculate_meteor(predicted_corpus_val, reference_corpus_val)
bert_score_ = calculate_bert_score(predicted_corpus_val, reference_corpus_val)

print(f'BLEU Score: {bleu_score_.score}')
print(f'BLEU-1: {bleu_score_.precisions[0]}')
print(f'BLEU-2: {bleu_score_.precisions[1]}')
print(f'BLEU-3: {bleu_score_.precisions[2]}')
print(f'BLEU-4: {bleu_score_.precisions[3]}')
print(f'METEOR Score: {meteor_score_}')
print(f'BERT Scores:')
print(f'Precision in BERT Score: {bert_score_[0]}')
print(f'Recall in BERT Score: {bert_score_[1]}')
print(f'F1 Score in BERT Score: {bert_score_[2]}')

/mnt/disk1/sumdev/CourseWork/a3_env/lib/python3.12/site-packages/transformers/generation/utils.py:1132: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


BLEU Score: 39.281465090051306
BLEU-1: 100.0
BLEU-2: 50.0
BLEU-3: 28.571428571428573
BLEU-4: 16.666666666666668
METEOR Score: 0.3697782590377402
BERT Scores:
Precision in BERT Score: 0.8268641829490662
Recall in BERT Score: 0.7662411332130432
F1 Score in BERT Score: 0.7942118644714355


In [14]:
reference_corpus_test, predicted_corpus_test = translate_engligh_to_german(test_dataset)

bleu_score_ = calculate_bleu(predicted_corpus_test, reference_corpus_test)
meteor_score_ = calculate_meteor(predicted_corpus_test, reference_corpus_test)
bert_score_ = calculate_bert_score(predicted_corpus_test, reference_corpus_test)

print(f'BLEU Score: {bleu_score_.score}')
print(f'BLEU-1: {bleu_score_.precisions[0]}')
print(f'BLEU-2: {bleu_score_.precisions[1]}')
print(f'BLEU-3: {bleu_score_.precisions[2]}')
print(f'BLEU-4: {bleu_score_.precisions[3]}')
print(f'METEOR Score: {meteor_score_}')
print(f'BERT Scores:')
print(f'Precision in BERT Score: {bert_score_[0]}')
print(f'Recall in BERT Score: {bert_score_[1]}')
print(f'F1 Score in BERT Score: {bert_score_[2]}')

BLEU Score: 0.0
BLEU-1: 66.66666666666667
BLEU-2: 25.0
BLEU-3: 25.0
BLEU-4: 0.0
METEOR Score: 0.3889479564070389
BERT Scores:
Precision in BERT Score: 0.8345420956611633
Recall in BERT Score: 0.7703850269317627
F1 Score in BERT Score: 0.8000085949897766


# Setup-2C

In [16]:
import torch
from transformers import T5ForConditionalGeneration, T5Tokenizer, T5Config, AutoTokenizer
from torch.utils.data import Dataset, DataLoader

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Load the T5 tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("google-t5/t5-small")
model_loaded = T5ForConditionalGeneration.from_pretrained("google-t5/t5-small")
model_loaded = model_loaded.to(device)
model_loaded.load_state_dict(torch.load('q2c_t5_model.pth'))

<All keys matched successfully>

In [17]:
import sacrebleu
from nltk.translate import meteor_score
from nltk.tokenize import word_tokenize
from bert_score import score
import nltk
nltk.download('wordnet')
nltk.download('punkt')


def calculate_bleu(predicted_corpus, reference_corpus):
    return sacrebleu.corpus_bleu(predicted_corpus, [[i] for i in reference_corpus])

def calculate_meteor(predicted_corpus, reference_corpus):
    m_score = 0
    for line in zip(predicted_corpus, reference_corpus):
        hypo = word_tokenize(line[0])
        ref = word_tokenize(line[1])
        # m_score += meteor_score.meteor_score([ref], hypo)
        m_score += meteor_score.single_meteor_score(ref, hypo)
    return m_score / len(reference_corpus)

def calculate_bert_score(predicted_corpus, reference_corpus):
    p, r, f1 = score(predicted_corpus, reference_corpus, lang="de")
    return p.mean().item(), r.mean().item(), f1.mean().item()

def translate_sentence(finetuned_model, dataset):
    finetuned_model = finetuned_model.to(device)
    sentence_german = []
    sentence_english = []
    sentence_english_translated = []
    for i in range(len(dataset)):
        sentence_german.append(dataset[i]['translation']['de'])
        sentence_english.append(dataset[i]['translation']['en'])
        input_ids = tokenizer(sentence_german[i], return_tensors="pt").input_ids
        input_ids = input_ids.to(device)
        outputs = finetuned_model.generate(input_ids)
        sentence_english_translated.append(tokenizer.decode(outputs[0], skip_special_tokens=True))
    return sentence_german, sentence_english, sentence_english_translated

[nltk_data] Downloading package wordnet to
[nltk_data]     /mnt/disk1/sumdev/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /mnt/disk1/sumdev/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [18]:
sentence_german_val, sentence_english_val, sentence_english_translated_val = translate_sentence(model_loaded, validation_dataset)

bleu_score_val = calculate_bleu(sentence_english_translated_val, sentence_english_val)
meteor_score_val = calculate_meteor(sentence_english_translated_val, sentence_english_val)
bert_score_val = calculate_bert_score(sentence_english_translated_val, sentence_english_val)

print(f'BLEU Score: {bleu_score_val.score}')
print(f'BLEU-1 Score: {bleu_score_val.precisions[0]}')
print(f'BLEU-2 Score: {bleu_score_val.precisions[1]}')
print(f'BLEU-3 Score: {bleu_score_val.precisions[2]}')
print(f'BLEU-4 Score: {bleu_score_val.precisions[3]}')
print(f'METEOR Score: {meteor_score_val}')
print(f'BERT Scores:')
print(f'Precision in BERT Score: {bert_score_val[0]}')
print(f'Recall in BERT Score: {bert_score_val[1]}')
print(f'F1 Score in BERT Score: {bert_score_val[2]}')

BLEU Score: 11.478744233307168
BLEU-1 Score: 70.0
BLEU-2 Score: 11.11111111111111
BLEU-3 Score: 6.25
BLEU-4 Score: 3.5714285714285716
METEOR Score: 0.3227082170618216
BERT Scores:
Precision in BERT Score: 0.79283207654953
Recall in BERT Score: 0.7551373243331909
F1 Score in BERT Score: 0.7728002071380615


In [19]:
sentence_german_test, sentence_english_test, sentence_english_translated_test = translate_sentence(model_loaded, test_dataset)

bleu_score_test = calculate_bleu(sentence_english_translated_test, sentence_english_test)
meteor_score_test = calculate_meteor(sentence_english_translated_test, sentence_english_test)
bert_score_test = calculate_bert_score(sentence_english_translated_test, sentence_english_test)

print(f'BLEU Score: {bleu_score_test.score}')
print(f'BLEU-1 Score: {bleu_score_test.precisions[0]}')
print(f'BLEU-2 Score: {bleu_score_test.precisions[1]}')
print(f'BLEU-3 Score: {bleu_score_test.precisions[2]}')
print(f'BLEU-4 Score: {bleu_score_test.precisions[3]}')
print(f'METEOR Score: {meteor_score_test}')
print(f'BERT Scores:')
print(f'Precision in BERT Score: {bert_score_test[0]}')
print(f'Recall in BERT Score: {bert_score_test[1]}')
print(f'F1 Score in BERT Score: {bert_score_test[2]}')

BLEU Score: 0.0
BLEU-1 Score: 66.66666666666667
BLEU-2 Score: 25.0
BLEU-3 Score: 25.0
BLEU-4 Score: 0.0
METEOR Score: 0.35205323785503273
BERT Scores:
Precision in BERT Score: 0.8002291321754456
Recall in BERT Score: 0.7628727555274963
F1 Score in BERT Score: 0.7803449034690857
